# Description
This notebook was used for the first round of experiments, which was non-iterative ablation. The results follow the same trends generally, but are less robust/reliable due to the fact that channel ablation only occured on the first timestep.

To see the code used for iterative ablation experiments, look at `setup.ipynb`

# Set up

In [2]:
%%time
# Import required libraries (Wall time: <1s)
import s3fs
import h5py
import numpy as np
import matplotlib.pyplot as plt
import subprocess
import time

CPU times: user 790 ms, sys: 405 ms, total: 1.2 s
Wall time: 792 ms


In [ ]:
%%time
# Install required dependencies (Wall time: 3min 15s) 
!pip install torch
!pip install torchvision
!pip install wandb
!conda install ruamel.yaml -y
!pip install timm
!pip install einops
!pip install apex

In [8]:
idxes = {'U10': 0, 'V10': 1, 'T2m': 2, 'sp': 3, 'mslp': 4, 'U1000': 5, 'V1000': 6, 
         'Z1000': 7, 'T850': 8, 'U850': 9, 'V850': 10, 'Z850': 11, 'RH850': 12, 'T500': 13, 
         'U500': 14, 'V500': 15, 'Z500': 16, 'RH500': 17, 'Z50': 18, 'TCWV': 19}

In [9]:
predict_variables = ['U10', 'V10', 'T2m', 'sp', 'mslp', 'Z1000', 'Z850', 'RH850']
ablate_variables = ['U10', 'V10', 'T2m', 'sp', 'mslp', 'U1000', 'V1000', 'Z1000', 'T850', 'U850', 'V850', 'Z850', 'RH850', 'T500', 'U500', 'V500', 'Z500', 'RH500', 'Z50', 'TCWV']

# Experiment

## Round 2 (iterative ablation)

In [5]:
ablate_variables = ['U10', 'V10', 'T2m', 'sp', 'mslp', 'U1000', 'V1000', 'Z1000', 'T850', 'U850', 'V850', 'Z850', 'RH850', 'T500', 'U500', 'V500', 'Z500', 'RH500', 'Z50', 'TCWV']
timesteps = 30

def format_time(seconds):
    minutes, seconds = divmod(seconds, 60)
    return f"{minutes:.0f} minutes {seconds:.2f} seconds"

# Record the overall start time
overall_start_time = time.time()

for i, ablate_var in enumerate(ablate_variables):
    start_time = time.time()
    print(f"Starting ablation {i+1}/{len(ablate_variables)} on {ablate_var}")

    command = f"python ../inference/inference_ablation_iterative.py \
        --config=afno_backbone \
        --run_num=0 \
        --weights 's3://climate-ai-data-science-sandbox-fourcastnet/FCN_weights_v0/backbone.ckpt' \
        --override_dir '../override_dir/iterative_ablation/results' \
        --vis \
        --target 'V10' \
        --ablate '{ablate_var}' \
        --timesteps {timesteps}"

    subprocess.run(command, shell=True, check=True)
    
    end_time = time.time()  # Record the end time for the inner loop
    elapsed_time = end_time - start_time  # Calculate the elapsed time for the inner loop
    formatted_time = format_time(elapsed_time)
    print(f"Ran ablation on {ablate_var} for {timesteps} timesteps in time: {formatted_time}")


# Calculate the total time for the entire script
overall_end_time = time.time()
overall_elapsed_time = overall_end_time - overall_start_time
formatted_overall_time = format_time(overall_elapsed_time)
print(f"Total time for the entire script: {formatted_overall_time}")

Starting ablation 1/20 on U10
Ran ablation on U10 for 30 timesteps in time: 0 minutes 0.00 seconds
Starting ablation 2/20 on V10
Ran ablation on V10 for 30 timesteps in time: 0 minutes 0.00 seconds
Starting ablation 3/20 on T2m
Ran ablation on T2m for 30 timesteps in time: 0 minutes 0.00 seconds
Starting ablation 4/20 on sp
Ran ablation on sp for 30 timesteps in time: 0 minutes 0.00 seconds
Starting ablation 5/20 on mslp
Ran ablation on mslp for 30 timesteps in time: 0 minutes 0.00 seconds
Starting ablation 6/20 on U1000
Ran ablation on U1000 for 30 timesteps in time: 0 minutes 0.00 seconds
Starting ablation 7/20 on V1000
Ran ablation on V1000 for 30 timesteps in time: 0 minutes 0.00 seconds
Starting ablation 8/20 on Z1000
Ran ablation on Z1000 for 30 timesteps in time: 0 minutes 0.00 seconds
Starting ablation 9/20 on T850
Ran ablation on T850 for 30 timesteps in time: 0 minutes 0.00 seconds
Starting ablation 10/20 on U850
Ran ablation on U850 for 30 timesteps in time: 0 minutes 0.00 s

## Round 1 (ablation non-iterative)

In [17]:
predict_variables = ['U10', 'V10', 'T2m', 'sp', 'mslp', 'Z1000', 'Z850', 'RH850']
ablate_variables = ['U10', 'V10', 'T2m', 'sp', 'mslp', 'U1000', 'V1000', 'Z1000', 'T850', 'U850', 'V850', 'Z850', 'RH850', 'T500', 'U500', 'V500', 'Z500', 'RH500', 'Z50', 'TCWV']
# predict_variables = ['U10', 'V10', 'T2m']
# ablate_variables = ['U10', 'V10', 'T2m']
timesteps = 30

def format_time(seconds):
    minutes, seconds = divmod(seconds, 60)
    return f"{minutes:.0f} minutes {seconds:.2f} seconds"

# Record the overall start time
overall_start_time = time.time()

for predict_var in predict_variables:
    outer_start_time = time.time()  # Record the start time for the outer loop
    print(f"Starting prediction on {predict_var}")
    
    for ablate_var in ablate_variables:
        # Don't do ablation on the prediction variable
        if predict_var is not ablate_var:
            start_time = time.time()  # Record the start time for the inner loop
            command = f"python FourCastNet/inference/inference_ablation.py \
                --config=afno_backbone \
                --run_num=0 \
                --weights 's3://climate-ai-data-science-sandbox-fourcastnet/FCN_weights_v0/backbone.ckpt' \
                --override_dir 'FourCastNet/override_dir/experiments' \
                --vis \
                --target '{predict_var}' \
                --ablate '{ablate_var}' \
                --timesteps {timesteps}"

            subprocess.run(command, shell=True, check=True)
            
            end_time = time.time()  # Record the end time for the inner loop
            elapsed_time = end_time - start_time  # Calculate the elapsed time for the inner loop
            formatted_time = format_time(elapsed_time)
            print(f"Run predict var {predict_var} with {ablate_var} ablated for {timesteps} timesteps in time: {formatted_time}")

    outer_end_time = time.time()  # Record the end time for the outer loop
    outer_elapsed_time = outer_end_time - outer_start_time  # Calculate the elapsed time for the outer loop
    formatted_outer_time = format_time(outer_elapsed_time)
    print(f" - - - - - - Done with prediction on {predict_var} - - - - - -")
    print(f"Elapsed time for prediction on {predict_var}: {formatted_outer_time}\n")

# Calculate the total time for the entire script
overall_end_time = time.time()
overall_elapsed_time = overall_end_time - overall_start_time
formatted_overall_time = format_time(overall_elapsed_time)
print(f"Total time for the entire script: {formatted_overall_time}")

Starting prediction on U10
Run predict var U10 with V10 ablated for 30 timesteps in time: 10 minutes 35.54 seconds
Run predict var U10 with T2m ablated for 30 timesteps in time: 10 minutes 24.49 seconds
Run predict var U10 with sp ablated for 30 timesteps in time: 10 minutes 15.64 seconds
Run predict var U10 with mslp ablated for 30 timesteps in time: 10 minutes 15.99 seconds
Run predict var U10 with U1000 ablated for 30 timesteps in time: 10 minutes 21.67 seconds
Run predict var U10 with V1000 ablated for 30 timesteps in time: 11 minutes 38.78 seconds
Run predict var U10 with Z1000 ablated for 30 timesteps in time: 10 minutes 42.66 seconds
Run predict var U10 with T850 ablated for 30 timesteps in time: 11 minutes 30.22 seconds
Run predict var U10 with U850 ablated for 30 timesteps in time: 10 minutes 27.84 seconds
Run predict var U10 with V850 ablated for 30 timesteps in time: 10 minutes 27.18 seconds
Run predict var U10 with Z850 ablated for 30 timesteps in time: 10 minutes 53.95 sec

In [16]:
# Mini experiment test 

import time
import subprocess

predict_variables = ['U10', 'V10', 'T2m']
ablate_variables = ['U10', 'V10', 'T2m']
timesteps = 3

def format_time(seconds):
    minutes, seconds = divmod(seconds, 60)
    return f"{minutes:.0f} minutes {seconds:.2f} seconds"

# Record the overall start time
overall_start_time = time.time()

for predict_var in predict_variables:
    outer_start_time = time.time()  # Record the start time for the outer loop
    print(f"Starting prediction on {predict_var}")
    
    for ablate_var in ablate_variables:
        # Don't do ablation on the prediction variable
        if predict_var is not ablate_var:
            start_time = time.time()  # Record the start time for the inner loop
#             command = f"python FourCastNet/inference/inference_ablation.py \
#                 --config=afno_backbone \
#                 --run_num=0 \
#                 --weights 's3://climate-ai-data-science-sandbox-fourcastnet/FCN_weights_v0/backbone.ckpt' \
#                 --override_dir 'FourCastNet/override_dir/experiments' \
#                 --vis \
#                 --target '{predict_var}' \
#                 --ablate '{ablate_var}' \
#                 --timesteps {timesteps}"

#             subprocess.run(command, shell=True, check=True)
            time.sleep(0.5)
            end_time = time.time()  # Record the end time for the inner loop
            elapsed_time = end_time - start_time  # Calculate the elapsed time for the inner loop
            formatted_time = format_time(elapsed_time)
            print(f"Run predict var {predict_var} with {ablate_var} ablated for {timesteps} timesteps in time: {formatted_time}")

    outer_end_time = time.time()  # Record the end time for the outer loop
    outer_elapsed_time = outer_end_time - outer_start_time  # Calculate the elapsed time for the outer loop
    formatted_outer_time = format_time(outer_elapsed_time)
    print(f" - - - - - - Done with prediction on {predict_var} - - - - - -")
    print(f"Elapsed time for prediction on {predict_var}: {formatted_outer_time}\n")

# Calculate the total time for the entire script
overall_end_time = time.time()
overall_elapsed_time = overall_end_time - overall_start_time
formatted_overall_time = format_time(overall_elapsed_time)
print(f"Total time for the entire script: {formatted_overall_time}")


Starting prediction on U10
Run predict var U10 with V10 ablated for 3 timesteps in time: 0 minutes 0.50 seconds
Run predict var U10 with T2m ablated for 3 timesteps in time: 0 minutes 0.50 seconds
 - - - - - - Done with prediction on U10 - - - - - -
Elapsed time for prediction on U10: 0 minutes 1.00 seconds

Starting prediction on V10
Run predict var V10 with U10 ablated for 3 timesteps in time: 0 minutes 0.50 seconds
Run predict var V10 with T2m ablated for 3 timesteps in time: 0 minutes 0.50 seconds
 - - - - - - Done with prediction on V10 - - - - - -
Elapsed time for prediction on V10: 0 minutes 1.00 seconds

Starting prediction on T2m
Run predict var T2m with U10 ablated for 3 timesteps in time: 0 minutes 0.50 seconds
Run predict var T2m with V10 ablated for 3 timesteps in time: 0 minutes 0.50 seconds
 - - - - - - Done with prediction on T2m - - - - - -
Elapsed time for prediction on T2m: 0 minutes 1.00 seconds

Total time for the entire script: 0 minutes 3.01 seconds


In [10]:
%%time
timesteps = 30

for predict_var in predict_variables:
    
    for ablate_var in ablate_variables:
        # Don't do ablation on the prediction variable
        if predict_var is not ablate_var:
            command = f"python FourCastNet/inference/inference_ablation.py \
                --config=afno_backbone \
                --run_num=0 \
                --weights 's3://climate-ai-data-science-sandbox-fourcastnet/FCN_weights_v0/backbone.ckpt' \
                --override_dir 'FourCastNet/override_dir/experiments' \
                --vis \
                --target '{predict_var}' \
                --ablate '{ablate_var}' \
                --timesteps {timesteps}"

            subprocess.run(command, shell=True, check=True)
            print(f"Run predict var {predict_var} with {ablate_var} ablated for {timesteps} timesteps in time {time}")

    print(f" - - - - - - Done with prediction on {predict_var} - - - - - - \n")

Run predict var U10 with V10 ablated for 30 timesteps
Run predict var U10 with T2m ablated for 30 timesteps
Run predict var U10 with sp ablated for 30 timesteps
Run predict var U10 with mslp ablated for 30 timesteps
Run predict var U10 with U1000 ablated for 30 timesteps
Run predict var U10 with V1000 ablated for 30 timesteps
Run predict var U10 with Z1000 ablated for 30 timesteps
Run predict var U10 with T850 ablated for 30 timesteps
Run predict var U10 with U850 ablated for 30 timesteps
Run predict var U10 with V850 ablated for 30 timesteps
Run predict var U10 with Z850 ablated for 30 timesteps
Run predict var U10 with RH850 ablated for 30 timesteps
Run predict var U10 with T500 ablated for 30 timesteps
Run predict var U10 with U500 ablated for 30 timesteps
Run predict var U10 with V500 ablated for 30 timesteps
Run predict var U10 with Z500 ablated for 30 timesteps
Run predict var U10 with RH500 ablated for 30 timesteps
Run predict var U10 with Z50 ablated for 30 timesteps
Run predic